In [1]:
from tls_crypt import *

In [2]:
aes = AES()
hkdf = HKDF()
x25519 = X25519()

In [3]:
def str2list(s) :
    r = []
    for c in s: r.append(ord(c))
    return r

def list2str(l) :
    r = ''
    for c in l: r += chr(c)
    return r

def hex2list(s):
    assert len(s) % 16 == 0
    l = []
    str = ''
    for c in s:
        str += c
        if len(str) == 2:
            l.append(int(str, 16))
            str = ''
    return l

def hexstr2list(s):
    li = s.split(' ')
    return [int(i, 16) for i in li]

def hexprint(li):
    print('[', end='')
    for i in li : print('{:02x}'.format(i), end=',')
    print('\b]')
    
def list2int(li):
    k = 0;
    for i in li:
        k *= 0x100;
        k += i
    return k

def int2list(k):
    r = []
    while k > 0:
        r.insert(0, k%0x100)
        k = k // 0x100
    return r

In [6]:
sha = SHA256()
a = sha.hash(str2list('aaaa1111'))

In [7]:
base64_encode(a)

'dUBo+TygkD4dt/CtPsWmFhecc49GKVndI4C24nQ2gNs='

In [4]:
client_key = 0x231a650ecfc578bea4b0467adaab33c2
server_key = 0x45ec5fe99527a6f2c9c3889fcd2e65c1
msg = '1400000c48037b71499792961515c436f4c6330f8748593625435272724d39632cdf4c060b0b0b0b0b0b0b0b0b0b0b0b'
iv = 0x4c63014c671a8bae25b22def0f57ce79

aes.key(server_key)
aes.iv(iv)
enc = aes.encrypt(hex2list(msg))
print(enc)
dec = aes.decrypt(enc[0])
print(dec)

([93, 66, 173, 31, 1, 161, 29, 215, 37, 51, 234, 157, 222, 85, 175, 189, 152, 225, 237, 39, 95, 229, 30, 179, 113, 246, 207, 89, 57, 83, 137, 212, 116, 201, 145, 197, 145, 53, 163, 142, 95, 156, 142, 36, 91, 42, 87, 223], [152, 91, 109, 166, 99, 242, 16, 53, 3, 76, 233, 136, 141, 108, 54, 180])
([20, 0, 0, 12, 72, 3, 123, 113, 73, 151, 146, 150, 21, 21, 196, 54, 244, 198, 51, 15, 135, 72, 89, 54, 37, 67, 82, 114, 114, 77, 57, 99, 44, 223, 76, 6, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11], [152, 91, 109, 166, 99, 242, 16, 53, 3, 76, 233, 136, 141, 108, 54, 180])


### client create an ephemeral x25519 key pair:
private key (32 octets): bf f9 11 88 28 38 46 dd 6a 21 34 ef 71
80 ca 2b 0b 14 fb 10 dc e7 07 b5 09 8c 0d dd c8 13 b2 df<br>
public key (32 octets): e4 ff b6 8a c0 5f 8d 96 c9 9d a2 66 98 34
6c 6b e1 64 82 ba dd da fe 05 1a 66 b4 f1 8d 66 8f 0b

In [5]:
pri = 0xbff91188283846dd6a2134ef7180ca2b0b14fb10dce707b5098c0dddc813b2df
pub = 0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b
print ('{0:#x}'.format(x25519.mul_g(pri)))

0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b


### Client extract secret "early":
salt: 0 (all zero octets)<br>
IKM (32 octets): 4e cd 0e b6 ec 3b 4d 87 f5 d6 02 8f 92 2c a4 c5
85 1a 27 7f d4 13 11 c9 e6 2d 2c 94 92 e1 c4 f3<br>
secret (32 octets): 9b 21 88 e9 b2 fc 6d 64 d7 1d c3 29 90 0e 20
bb 41 91 50 00 f6 78 aa 83 9c bb 79 7c b7 d8 33 2c

In [6]:
ikm = [0x4e,0xcd,0x0e,0xb6,0xec,0x3b,0x4d,0x87,0xf5,0xd6,0x02,0x8f,0x92,0x2c,0xa4,0xc5,
          0x85,0x1a,0x27,0x7f,0xd4,0x13,0x11,0xc9,0xe6,0x2d,0x2c,0x94,0x92,0xe1,0xc4,0xf3]
hkdf.zero_salt()
early = hkdf.extract(ikm)
hexprint(early)

[9b,21,88,e9,b2,fc,6d,64,d7,1d,c3,29,90,0e,20,bb,41,91,50,00,f6,78,aa,83,9c,bb,79,7c,b7,d8,33,2c,]


### client construct a ClientHello handshake message:
ClientHello (477 octets):<br>
  01
 00
 01
 fc
 03
 03
 1b
 c3
 ce
 b6
 bb
 e3
 9c
ff 93 83
 55 b5 a5 0a db
 6d
 b2
 1b
 7a
 6a
 f6
 49
 d7
 b4
 bc
 41
 9d
 78
76 48 7d
 95 00 00 06 13
 01
 13
 03
 13
 02
 01
 00
 01
 cd
 00
 00
 00
 0b
00 09 00
 00 06 73 65 72
 76
 65
 72
 ff
 01
 00
 01
 00
 00
 0a
 00
 14
 00
12 00 1d
 00 17 00 18 00
 19
 01
 00
 01
 01
 01
 02
 01
 03
 01
 04
 00
 33
00 26 00
 24 00 1d 00 20
 e4
 ff
 b6
 8a
 c0
 5f
 8d
 96
 c9
 9d
 a2
 66
 98
34 6c 6b
 e1 64 82 ba dd
 da
 fe
 05
 1a
 66
 b4
 f1
 8d
 66
 8f
 0b
 00
 2a
00 00 00
 2b 00 03 02 03
 04
 00
 0d
 00
 20
 00
 1e
 04
 03
 05
 03
 06
 03
02 03 08
 04 08 05 08 06
 04
 01
 05
 01
 06
 01
 02
 01
 04
 02
 05
 02
 06
02
 02
 02
 00
 2d
 00
 02
 01
 01
 00
 1c
 00
 02
 40
 01
 00
 15
 00
 57
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 29
 00
 dd
 00
 b8
 00
 b2
 2c
 03
 5d
 82
 93
 59
 ee
 5f
 f7
 af
 4e
 c9
00
 00
 00
 00
 26
 2a
 64
 94
 dc
 48
 6d
 2c
 8a
 34
 cb
 33
 fa
 90
 bf
 1b
 00
70
 ad
 3c
 49
 88
 83
 c9
 36
 7c
 09
 a2
 be
 78
 5a
 bc
 55
 cd
 22
 60
 97
 a3
a9
 82
 11
 72
 83
 f8
 2a
 03
 a1
 43
 ef
 d3
 ff
 5d
 d3
 6d
 64
 e8
 61
 be
 7f
d6
 1d
 28
 27
 db
 27
 9c
 ce
 14
 50
 77
 d4
 54
 a3
 66
 4d
 4e
 6d
 a4
 d2
 9e
e0
 37
 25
 a6
 a4
 da
 fc
 d0
 fc
 67
 d2
 ae
 a7
 05
 29
 51
 3e
 3d
 a2
 67
 7f
a5
 90
 6c
 5b
 3f
 7d
 8f
 92
 f2
 28
 bd
 a4
 0d
 da
 72
 14
 70
 f9
 fb
 f2
 97
b5
 ae
 a6
 17
 64
 6f
 ac
 5c
 03
 27
 2e
 97
 07
 27
 c6
 21
 a7
 91
 41
 ef
 5f
7d
 e6
 50
 5e
 5b
 fb
 c3
 88
 e9
 33
 43
 69
 40
 93
 93
 4a
 e4
 d3
 57
 fa
 d6
aa
 cb


In [7]:
client_hello = '01 00 01 fc 03 03 1b c3 ce b6 bb e3 9c ff 93 83 55 b5 a5 0a db 6d b2 1b 7a 6a f6 49 d7 b4 bc 41 9d 78 76 48 7d 95 00 00 06 13 01 13 03 13 02 01 00 01 cd 00 00 00 0b 00 09 00 00 06 73 65 72 76 65 72 ff 01 00 01 00 00 0a 00 14 00 12 00 1d 00 17 00 18 00 19 01 00 01 01 01 02 01 03 01 04 00 33 00 26 00 24 00 1d 00 20 e4 ff b6 8a c0 5f 8d 96 c9 9d a2 66 98 34 6c 6b e1 64 82 ba dd da fe 05 1a 66 b4 f1 8d 66 8f 0b 00 2a 00 00 00 2b 00 03 02 03 04 00 0d 00 20 00 1e 04 03 05 03 06 03 02 03 08 04 08 05 08 06 04 01 05 01 06 01 02 01 04 02 05 02 06 02 02 02 00 2d 00 02 01 01 00 1c 00 02 40 01 00 15 00 57 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 29 00 dd 00 b8 00 b2 2c 03 5d 82 93 59 ee 5f f7 af 4e c9 00 00 00 00 26 2a 64 94 dc 48 6d 2c 8a 34 cb 33 fa 90 bf 1b 00 70 ad 3c 49 88 83 c9 36 7c 09 a2 be 78 5a bc 55 cd 22 60 97 a3 a9 82 11 72 83 f8 2a 03 a1 43 ef d3 ff 5d d3 6d 64 e8 61 be 7f d6 1d 28 27 db 27 9c ce 14 50 77 d4 54 a3 66 4d 4e 6d a4 d2 9e e0 37 25 a6 a4 da fc d0 fc 67 d2 ae a7 05 29 51 3e 3d a2 67 7f a5 90 6c 5b 3f 7d 8f 92 f2 28 bd a4 0d da 72 14 70 f9 fb f2 97 b5 ae a6 17 64 6f ac 5c 03 27 2e 97 07 27 c6 21 a7 91 41 ef 5f 7d e6 50 5e 5b fb c3 88 e9 33 43 69 40 93 93 4a e4 d3 57 fa d6 aa cb'
li = hexstr2list(client_hello)
hexprint(li)

[01,00,01,fc,03,03,1b,c3,ce,b6,bb,e3,9c,ff,93,83,55,b5,a5,0a,db,6d,b2,1b,7a,6a,f6,49,d7,b4,bc,41,9d,78,76,48,7d,95,00,00,06,13,01,13,03,13,02,01,00,01,cd,00,00,00,0b,00,09,00,00,06,73,65,72,76,65,72,ff,01,00,01,00,00,0a,00,14,00,12,00,1d,00,17,00,18,00,19,01,00,01,01,01,02,01,03,01,04,00,33,00,26,00,24,00,1d,00,20,e4,ff,b6,8a,c0,5f,8d,96,c9,9d,a2,66,98,34,6c,6b,e1,64,82,ba,dd,da,fe,05,1a,66,b4,f1,8d,66,8f,0b,00,2a,00,00,00,2b,00,03,02,03,04,00,0d,00,20,00,1e,04,03,05,03,06,03,02,03,08,04,08,05,08,06,04,01,05,01,06,01,02,01,04,02,05,02,06,02,02,02,00,2d,00,02,01,01,00,1c,00,02,40,01,00,15,00,57,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,29,00,dd,00,b8,00,b2,2c,03,5d,82,93,59,ee,5f,f7,af,4e,c9,00,00,00,00,26,2a,64,94,dc,48,6d,2c,8a,34,cb,33,fa,90,bf,1b,00,70,ad,3c,49,88,

### client calculate PSK binder:
ClientHello
 prefix (477
 octets):
 01 00 01 fc 03 03 1b c3 ce b6 bb
e3 9c ff
... 上同

binder hash (32 octets): 63 22 4b 2e 45 73 f2 d3 45 4c a8 4b 9d
00 9a 04 f6 be 9e 05 71 1a 83 96 47 3a ef a0 1e 92 4a 14<br>
PRK (32 octets): 69 fe 13 1a 3b ba d5 d6 3c 64 ee bc c3 0e 39 5b
9d 81 07 72 6a 13 d0 74 e3 89 db c8 a4 e4 72 56
hash (0 octets):
 (empty)<br>
info (18 octets):
 00 20 0e 74 6c 73 31 33 20 66 69 6e 69 73 68 65
64 00<br>
expanded (32 octets): 55 88 67 3e 72 cb 59 c8 7d 22 0c af fe 94
f2 de a9 a3 b1 60 9f 7d 50 e9 0a 48 22 7d b9 ed 7e aa<br>
finished (32 octets): 3a dd 4f b2 d8 fd f8 22 a0 ca 3c f7 67 8e
f5 e8 8d ae 99 01 41 c5 92 4d 57 bb 6f a3 1b 9e 5f 9d


In [8]:
sha = SHA256()
tmp = sha.hash(li)
hexprint(tmp)
hkdf.salt(early)
binder_key = hkdf.derive_secret('res binder', [])
hexprint(binder_key)
hkdf.salt(binder_key)
expanded = hkdf.expand_label('finished', '', 32)
hexprint(expanded)
hkdf.salt(expanded)
finished = hkdf.hash(tmp)
hexprint(finished)

[63,22,4b,2e,45,73,f2,d3,45,4c,a8,4b,9d,00,9a,04,f6,be,9e,05,71,1a,83,96,47,3a,ef,a0,1e,92,4a,14,]
[69,fe,13,1a,3b,ba,d5,d6,3c,64,ee,bc,c3,0e,39,5b,9d,81,07,72,6a,13,d0,74,e3,89,db,c8,a4,e4,72,56,]
[55,88,67,3e,72,cb,59,c8,7d,22,0c,af,fe,94,f2,de,a9,a3,b1,60,9f,7d,50,e9,0a,48,22,7d,b9,ed,7e,aa,]
[3a,dd,4f,b2,d8,fd,f8,22,a0,ca,3c,f7,67,8e,f5,e8,8d,ae,99,01,41,c5,92,4d,57,bb,6f,a3,1b,9e,5f,9d,]


### client send handshake record:
payload (512 octets): 01 00 01 fc 03 03 1b c3 ce b6 bb e3 9c ff
93 83 55 b5 a5 0a db 6d b2 1b 7a 6a f6 49 d7 b4 bc 41 9d 78 76
48 7d 95 00 00 06 13 01 13 03 13 02 01 00 01 cd 00 00 00 0b 00
09 00 00 06 73 65 72 76 65 72 ff 01 00 01 00 00 0a 00 14 00 12
00 1d 00 17 00 18 00 19 01 00 01 01 01 02 01 03 01 04 00 33 00
26 00 24 00 1d 00 20 e4 ff b6 8a c0 5f 8d 96 c9 9d a2 66 98 34
6c 6b e1 64 82 ba dd da fe 05 1a 66 b4 f1 8d 66 8f 0b 00 2a 00
00 00 2b 00 03 02 03 04 00 0d 00 20 00 1e 04 03 05 03 06 03 02
03 08 04 08 05 08 06 04 01 05 01 06 01 02 01 04 02 05 02 06 02
02 02 00 2d 00 02 01 01 00 1c 00 02 40 01 00 15 00 57 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 29 00 dd 00 b8 00 b2 2c 03 5d 82 93 59 ee 5f f7 af 4e c9 00
00 00 00 26 2a 64 94 dc 48 6d 2c 8a 34 cb 33 fa 90 bf 1b 00 70
ad 3c 49 88 83 c9 36 7c 09 a2 be 78 5a bc 55 cd 22 60 97 a3 a9
82 11 72 83 f8 2a 03 a1 43 ef d3 ff 5d d3 6d 64 e8 61 be 7f d6
1d 28 27 db 27 9c ce 14 50 77 d4 54 a3 66 4d 4e 6d a4 d2 9e e0
37 25 a6 a4 da fc d0 fc 67 d2 ae a7 05 29 51 3e 3d a2 67 7f a5
90 6c 5b 3f 7d 8f 92 f2 28 bd a4 0d da 72 14 70 f9 fb f2 97 b5
ae a6 17 64 6f ac 5c 03 27 2e 97 07 27 c6 21 a7 91 41 ef 5f 7d
e6 50 5e 5b fb c3 88 e9 33 43 69 40 93 93 4a e4 d3 57 fa d6 aa
cb 00 21 20 3a dd 4f b2 d8 fd f8 22 a0 ca 3c f7 67 8e f5 e8 8d
ae 99 01 41 c5 92 4d 57 bb 6f a3 1b 9e 5f 9d<br>
complete
 record (517
 octets):
 16 03 01 02 00 01 00 01 fc 03 03 1b
c3 ce
 b6 bb e3 9c
 ff 93 83
 55 b5 a5 0a db 6d b2 1b 7a 6a f6 49
d7 b4
 bc 41 9d 78
 76 48 7d
 95 00 00 06 13 01 13 03 13 02 01 00
01 cd
 00 00 00 0b
 00 09 00
 00 06 73 65 72 76 65 72 ff 01 00 01
00 00
 0a 00 14 00
 12 00 1d
 00 17 00 18 00 19 01 00 01 01 01 02
01 03
 01 04 00 33
 00 26 00
 24 00 1d 00 20 e4 ff b6 8a c0 5f 8d
96 c9
 9d a2 66 98
 34 6c 6b
 e1 64 82 ba dd da fe 05 1a 66 b4 f1
8d 66
 8f 0b 00 2a
 00 00 00
 2b 00 03 02 03 04 00 0d 00 20 00 1e
04 03
 05 03 06 03
 02 03 08
 04 08 05 08 06 04 01 05 01 06 01 02
01
 04
 02
 05
 02
 06
 02
 02
 02
 00
 2d
 00
 02
 01
 01
 00
 1c
 00
 02
 40
 01
00
 15
 00
 57
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
 00
00
 00
 00
 00
 00
 00
 00
 00
 29
 00
 dd
 00
 b8
 00
 b2
 2c
 03
 5d
 82
 93
 59
ee
 5f
 f7
 af
 4e
 c9
 00
 00
 00
 00
 26
 2a
 64
 94
 dc
 48
 6d
 2c
 8a
 34
 cb
33
 fa
 90
 bf
 1b
 00
 70
 ad
 3c
 49
 88
 83
 c9
 36
 7c
 09
 a2
 be
 78
 5a
 bc
55
 cd
 22
 60
 97
 a3
 a9
 82
 11
 72
 83
 f8
 2a
 03
 a1
 43
 ef
 d3
 ff
 5d
 d3
6d
 64
 e8
 61
 be
 7f
 d6
 1d
 28
 27
 db
 27
 9c
 ce
 14
 50
 77
 d4
 54
 a3
 66
4d
 4e
 6d
 a4
 d2
 9e
 e0
 37
 25
 a6
 a4
 da
 fc
 d0
 fc
 67
 d2
 ae
 a7
 05
 29
51
 3e
 3d
 a2
 67
 7f
 a5
 90
 6c
 5b
 3f
 7d
 8f
 92
 f2
 28
 bd
 a4
 0d
 da
 72
14
 70
 f9
 fb
 f2
 97
 b5
 ae
 a6
 17
 64
 6f
 ac
 5c
 03
 27
 2e
 97
 07
 27
 c6
21
 a7
 91
 41
 ef
 5f
 7d
 e6
 50
 5e
 5b
 fb
 c3
 88
 e9
 33
 43
 69
 40
 93
 93
4a
 e4
 d3
 57
 fa
 d6
 aa
 cb
 00
 21
 20
 3a
 dd
 4f
 b2
 d8
 fd
 f8
 22
 a0
 ca
3c
 f7
 67
 8e
 f5
 e8
 8d
 ae
 99
 01
 41
 c5
 92
 4d
 57
 bb
 6f
 a3
 1b
 9e
 5f
9d


In [9]:
payload = li + [0, 0x21, 0x20] + finished
hexprint(payload)

[01,00,01,fc,03,03,1b,c3,ce,b6,bb,e3,9c,ff,93,83,55,b5,a5,0a,db,6d,b2,1b,7a,6a,f6,49,d7,b4,bc,41,9d,78,76,48,7d,95,00,00,06,13,01,13,03,13,02,01,00,01,cd,00,00,00,0b,00,09,00,00,06,73,65,72,76,65,72,ff,01,00,01,00,00,0a,00,14,00,12,00,1d,00,17,00,18,00,19,01,00,01,01,01,02,01,03,01,04,00,33,00,26,00,24,00,1d,00,20,e4,ff,b6,8a,c0,5f,8d,96,c9,9d,a2,66,98,34,6c,6b,e1,64,82,ba,dd,da,fe,05,1a,66,b4,f1,8d,66,8f,0b,00,2a,00,00,00,2b,00,03,02,03,04,00,0d,00,20,00,1e,04,03,05,03,06,03,02,03,08,04,08,05,08,06,04,01,05,01,06,01,02,01,04,02,05,02,06,02,02,02,00,2d,00,02,01,01,00,1c,00,02,40,01,00,15,00,57,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,29,00,dd,00,b8,00,b2,2c,03,5d,82,93,59,ee,5f,f7,af,4e,c9,00,00,00,00,26,2a,64,94,dc,48,6d,2c,8a,34,cb,33,fa,90,bf,1b,00,70,ad,3c,49,88,

### client derive secret "tls13 c e traffic":
PRK (32 octets): 9b 21 88 e9 b2 fc 6d 64 d7 1d c3 29 90 0e 20 bb
41 91 50 00 f6 78 aa 83 9c bb 79 7c b7 d8 33 2c<br>
hash (32 octets): 08 ad 0f a0 5d 7c 72 33 b1 77 5b a2 ff 9f 4c 5b
8b 59 27 6b 7f 22 7f 13 a9 76 24 5f 5d 96 09 13<br>
info (53 octets): 00 20 11 74 6c 73 31 33 20 63 20 65 20 74 72 61
66 66 69 63 20 08 ad 0f a0 5d 7c 72 33 b1 77 5b a2 ff 9f 4c 5b
8b 59 27 6b 7f 22 7f 13 a9 76 24 5f 5d 96 09 13<br>
expanded (32 octets): 3f bb e6 a6 0d eb 66 c3 0a 32 79 5a ba 0e
ff 7e aa 10 10 55 86 e7 be 5c 09 67 8d 63 b6 ca ab 62<br>


In [10]:
hkdf.salt(early)
c_e_traffic = hkdf.derive_secret('c e traffic', payload)
hexprint(c_e_traffic)

[3f,bb,e6,a6,0d,eb,66,c3,0a,32,79,5a,ba,0e,ff,7e,aa,10,10,55,86,e7,be,5c,09,67,8d,63,b6,ca,ab,62,]


### client derive secret "tls13 e exp master":
PRK (32 octets): 9b 21 88 e9 b2 fc 6d 64 d7 1d c3 29 90 0e 20 bb
41 91 50 00 f6 78 aa 83 9c bb 79 7c b7 d8 33 2c<br>
hash (32 octets): 08 ad 0f a0 5d 7c 72 33 b1 77 5b a2 ff 9f 4c 5b
8b 59 27 6b 7f 22 7f 13 a9 76 24 5f 5d 96 09 13<br>
info (54 octets): 00 20 12 74 6c 73 31 33 20 65 20 65 78 70 20 6d
61 73 74 65 72 20 08 ad 0f a0 5d 7c 72 33 b1 77 5b a2 ff 9f 4c
5b 8b 59 27 6b 7f 22 7f 13 a9 76 24 5f 5d 96 09 13<br>
expanded (32 octets): b2 02 68 66 61 09 37 d7 42 3e 5b e9 08 62
cc f2 4c 0e 60 91 18 6d 34 f8 12 08 9f f5 be 2e f7 df<br>


In [11]:
e_exp_master = hkdf.derive_secret('e exp master', payload)
hexprint(e_exp_master)

[b2,02,68,66,61,09,37,d7,42,3e,5b,e9,08,62,cc,f2,4c,0e,60,91,18,6d,34,f8,12,08,9f,f5,be,2e,f7,df,]


### client derive write traffic keys for early application data:
PRK (32 octets): 3f bb e6 a6 0d eb 66 c3 0a 32 79 5a ba 0e ff 7e
aa 10 10 55 86 e7 be 5c 09 67 8d 63 b6 ca ab 62<br>
key info (13 octets):
 00 10 09 74 6c 73 31 33 20 6b 65 79 00<br>
key expanded (16 octets):
 92 02 05 a5 b7 bf 21 15 e6 fc 5c 29 42
83 4f 54<br>
iv info (12 octets):
 00 0c 08 74 6c 73 31 33 20 69 76 00<br>
iv expanded (12 octets):
 6d 47 5f 09 93 c8 e5 64 61 0d b2 b9

In [12]:
hkdf.salt(c_e_traffic)
c_key = hkdf.expand_label('key', '', 16)
c_iv = hkdf.expand_label('iv', '', 12)
hexprint(c_key)
hexprint(c_iv)

[92,02,05,a5,b7,bf,21,15,e6,fc,5c,29,42,83,4f,54,]
[6d,47,5f,09,93,c8,e5,64,61,0d,b2,b9,]


### client send application_data record:
payload (6 octets):
 41 42 43 44 45 46<br>
complete record (28 octets): 17 03 03 00 17 ab 1d f4 20 e7 5c 45
7a 7c c5 d2 84 4f 76 d5 ae e4 b4 ed bf 04 9b e0


In [13]:
p = [0x41, 0x42, 0x43, 0x44, 0x45, 0x46, 23]#record 23
aes.key(list2int(c_key))
aes.iv(list2int(c_iv))
aes.aad([0x17, 3,3,0, 0x17])
for i in aes.encrypt(p): hexprint(i)


[ab,1d,f4,20,e7,5c,45,]
[7a,7c,c5,d2,84,4f,76,d5,ae,e4,b4,ed,bf,04,9b,e0,]


### server extract secret "early" (same as client early secret)
 calculate PSK binder (same as client):<br>
 create an ephemeral x25519 key pair:<br>
private key (32 octets): de 5b 44 76 e7 b4 90 b2 65 2d 33 8a cb
f2 94 80 66 f2 55 f9 44 0e 23 b9 8f c6 98 35 29 8d c1 07<br>
public key (32 octets): 12 17 61 ee 42 c3 33 e1 b9 e7 7b 60 dd 57
c2 05 3c d9 45 12 ab 47 f1 15 e8 6e ff 50 94 2c ea 31


In [14]:
s_pri = list2int(hexstr2list('de 5b 44 76 e7 b4 90 b2 65 2d 33 8a cb f2 94 80 66 f2 55 f9 44 0e 23 b9 8f c6 98 35 29 8d c1 07'))
s_pub = x25519.mul_g(s_pri)
print('{:x}'.format(s_pub))

121761ee42c333e1b9e77b60dd57c2053cd94512ab47f115e86eff50942cea31


### server derive secret "tls13 c e traffic" (same as client)
{server}
 derive secret "tls13 e exp master" (same as client)<br>
{server}
 construct a ServerHello handshake message:<br>
ServerHello
 (96 octets): 02 00 00 5c 03 03 3c cf d2 de c8 90 22
27 63 47
 2a e8 13 67 77 c9 d7 35 87 77 bb 66 e9 1e a5 12 24 95
f5 59 ea
 2d 00 13 01 00 00 34 00 29 00 02 00 00 00 33 00 24 00
1d 00 20
 12 17 61 ee 42 c3 33 e1 b9 e7 7b 60 dd 57 c2 05 3c d9
45 12 ab
 47 f1 15 e8 6e ff 50 94 2c ea 31 00 2b 00 02 03 04


### server derive secret for handshake "tls13 derived":
PRK (32 octets): 9b 21 88 e9 b2 fc 6d 64 d7 1d c3 29 90 0e 20 bb
41 91 50 00 f6 78 aa 83 9c bb 79 7c b7 d8 33 2c<br>
hash (32 octets): e3 b0 c4 42 98 fc 1c 14 9a fb f4 c8 99 6f b9 24
27 ae 41 e4 64 9b 93 4c a4 95 99 1b 78 52 b8 55<br>
info (49 octets): 00 20 0d 74 6c 73 31 33 20 64 65 72 69 76 65 64
20 e3 b0 c4 42 98 fc 1c 14 9a fb f4 c8 99 6f b9 24 27 ae 41 e4
64 9b 93 4c a4 95 99 1b 78 52 b8 55<br>
expanded (32 octets): 5f 17 90 bb d8 2c 5e 7d 37 6e d2 e1 e5 2f
8e 60 38 c9 34 6d b6 1b 43 be 9a 52 f7 7e f3 99 8e 80


In [15]:
hkdf.salt(early)
handshake_secret = hkdf.derive_secret('derived', [])
hexprint(handshake_secret)

[5f,17,90,bb,d8,2c,5e,7d,37,6e,d2,e1,e5,2f,8e,60,38,c9,34,6d,b6,1b,43,be,9a,52,f7,7e,f3,99,8e,80,]


### server extract secret "handshake":
salt (32 octets): 5f 17 90 bb d8 2c 5e 7d 37 6e d2 e1 e5 2f 8e 60
38 c9 34 6d b6 1b 43 be 9a 52 f7 7e f3 99 8e 80<br>
IKM (32 octets): f4 41 94 75 6f f9 ec 9d 25 18 06 35 d6 6e a6 82
4c 6a b3 bf 17 99 77 be 37 f7 23 57 0e 7c cb 2e<br>
secret (32 octets): 00 5c b1 12 fd 8e b4 cc c6 23 bb 88 a0 7c 64
b3 ed e1 60 53 63 fc 7d 0d f8 c7 ce 4f f0 fb 4a e6

In [16]:
key = x25519.mul(s_pri, pub)
print('{:x}'.format(key))
hkdf.salt(handshake_secret)
handshake_s = hkdf.extract(int2list(key))
hexprint(handshake_s)

f44194756ff9ec9d25180635d66ea6824c6ab3bf179977be37f723570e7ccb2e
[00,5c,b1,12,fd,8e,b4,cc,c6,23,bb,88,a0,7c,64,b3,ed,e1,60,53,63,fc,7d,0d,f8,c7,ce,4f,f0,fb,4a,e6]


### server derive secret "tls13 c hs traffic":
PRK (32 octets): 00 5c b1 12 fd 8e b4 cc c6 23 bb 88 a0 7c 64 b3
ed e1 60 53 63 fc 7d 0d f8 c7 ce 4f f0 fb 4a e6<br>
hash (32 octets): f7 36 cb 34 fe 25 e7 01 55 1b ee 6f d2 4c 1c c7
10 2a 7d af 94 05 cb 15 d9 7a af e1 6f 75 7d 03<br>
info (54 octets): 00 20 12 74 6c 73 31 33 20 63 20 68 73 20 74 72
61 66 66 69 63 20 f7 36 cb 34 fe 25 e7 01 55 1b ee 6f d2 4c 1c
c7 10 2a 7d af 94 05 cb 15 d9 7a af e1 6f 75 7d 03<br>
expanded (32 octets): 2f aa c0 8f 85 1d 35 fe a3 60 4f cb 4d e8
2d c6 2c 9b 16 4a 70 97 4d 04 62 e2 7f 1a b2 78 70 0f


In [17]:
server_h = '02 00 00 5c 03 03 3c cf d2 de c8 90 22 27 63 47 2a e8 13 67 77 c9 d7 35 87 77 bb 66 e9 1e a5 12 24 95 f5 59 ea 2d 00 13 01 00 00 34 00 29 00 02 00 00 00 33 00 24 00 1d 00 20 12 17 61 ee 42 c3 33 e1 b9 e7 7b 60 dd 57 c2 05 3c d9 45 12 ab 47 f1 15 e8 6e ff 50 94 2c ea 31 00 2b 00 02 03 04'
server_hello = hexstr2list(server_h)
hkdf.salt(handshake_s)
c_hs_traffic = hkdf.derive_secret('c hs traffic', payload + server_hello)
hexprint(c_hs_traffic)

[2f,aa,c0,8f,85,1d,35,fe,a3,60,4f,cb,4d,e8,2d,c6,2c,9b,16,4a,70,97,4d,04,62,e2,7f,1a,b2,78,70,0f]


### server derive secret "tls13 s hs traffic":
PRK (32 octets): 00 5c b1 12 fd 8e b4 cc c6 23 bb 88 a0 7c 64 b3
ed e1 60 53 63 fc 7d 0d f8 c7 ce 4f f0 fb 4a e6<br>
hash (32 octets): f7 36 cb 34 fe 25 e7 01 55 1b ee 6f d2 4c 1c c7
10 2a 7d af 94 05 cb 15 d9 7a af e1 6f 75 7d 03<br>
info (54 octets): 00 20 12 74 6c 73 31 33 20 73 20 68 73 20 74 72
61 66 66 69 63 20 f7 36 cb 34 fe 25 e7 01 55 1b ee 6f d2 4c 1c
c7 10 2a 7d af 94 05 cb 15 d9 7a af e1 6f 75 7d 03<br>
expanded (32 octets): fe 92 7a e2 71 31 2e 8b f0 27 5b 58 1c 54
ee f0 20 45 0d c4 ec ff aa 05 a1 a3 5d 27 51 8e 78 03

In [18]:
hkdf.salt(handshake_s)
s_hs_traffic = hkdf.derive_secret('s hs traffic', payload+server_hello)
hexprint(s_hs_traffic)

[fe,92,7a,e2,71,31,2e,8b,f0,27,5b,58,1c,54,ee,f0,20,45,0d,c4,ec,ff,aa,05,a1,a3,5d,27,51,8e,78,03]


### server derive secret for master "tls13 derived":
PRK (32 octets): 00 5c b1 12 fd 8e b4 cc c6 23 bb 88 a0 7c 64 b3
ed e1 60 53 63 fc 7d 0d f8 c7 ce 4f f0 fb 4a e6<br>
hash (32 octets): e3 b0 c4 42 98 fc 1c 14 9a fb f4 c8 99 6f b9 24
27 ae 41 e4 64 9b 93 4c a4 95 99 1b 78 52 b8 55<br>
info (49 octets): 00 20 0d 74 6c 73 31 33 20 64 65 72 69 76 65 64
20 e3 b0 c4 42 98 fc 1c 14 9a fb f4 c8 99 6f b9 24 27 ae 41 e4
64 9b 93 4c a4 95 99 1b 78 52 b8 55<br>
expanded (32 octets): e2 f1 60 30 25 1d f0 87 4b a1 9b 9a ba 25
76 10 bc 6d 53 1c 1d d2 06 df 0c a6 e8 4a e2 a2 67 42

In [19]:
dr = hkdf.derive_secret('derived', [])
hexprint(dr)

[e2,f1,60,30,25,1d,f0,87,4b,a1,9b,9a,ba,25,76,10,bc,6d,53,1c,1d,d2,06,df,0c,a6,e8,4a,e2,a2,67,42,]


### server extract secret "master":
salt (32 octets): e2 f1 60 30 25 1d f0 87 4b a1 9b 9a ba 25 76 10
bc 6d 53 1c 1d d2 06 df 0c a6 e8 4a e2 a2 67 42<br>
IKM (32 octets): 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00<br>
secret (32 octets): e2 d3 2d 4e d6 6d d3 78 97 a0 e8 0c 84 10 75
03 ce 58 bf 8a ad 4c b5 5a 50 02 d7 7e cb 89 0e ce

In [20]:
hkdf.salt(dr)
master = hkdf.extract([0]*32)
hexprint(master)

[e2,d3,2d,4e,d6,6d,d3,78,97,a0,e8,0c,84,10,75,03,ce,58,bf,8a,ad,4c,b5,5a,50,02,d7,7e,cb,89,0e,ce,]


### server send handshake record:
payload (96
 octets):
 02 00 00 5c 03 03 3c cf d2 de c8 90 22 27 63
47 2a e8
 13 67 77
 c9 d7 35 87 77 bb 66 e9 1e a5 12 24 95 f5 59
ea 2d 00
 13 01 00
 00 34 00 29 00 02 00 00 00 33 00 24 00 1d 00
20 12 17
 61 ee 42
 c3 33 e1 b9 e7 7b 60 dd 57 c2 05 3c d9 45 12
ab 47 f1
 15 e8 6e
 ff 50 94 2c ea 31 00 2b 00 02 03 04<br>
complete
 record (101
 octets):
 16 03 03 00 60 02 00 00 5c 03 03 3c
cf d2
 de c8 90 22
 27 63 47
 2a e8 13 67 77 c9 d7 35 87 77 bb 66
e9 1e
 a5 12 24 95
 f5 59 ea
 2d 00 13 01 00 00 34 00 29 00 02 00
00 00
 33 00 24 00
 1d 00 20
 12 17 61 ee 42 c3 33 e1 b9 e7 7b 60
dd 57
 c2 05 3c d9
 45 12 ab
 47 f1 15 e8 6e ff 50 94 2c ea 31 00
2b 00
 02 03 04


In [21]:
server_h = '02 00 00 5c 03 03 3c cf d2 de c8 90 22 27 63 47 2a e8 13 67 77 c9 d7 35 87 77 bb 66 e9 1e a5 12 24 95 f5 59 ea 2d 00 13 01 00 00 34 00 29 00 02 00 00 00 33 00 24 00 1d 00 20 12 17 61 ee 42 c3 33 e1 b9 e7 7b 60 dd 57 c2 05 3c d9 45 12 ab 47 f1 15 e8 6e ff 50 94 2c ea 31 00 2b 00 02 03 04'
server_hello = hexstr2list(server_h)

### server derive write traffic keys for handshake data:
PRK (32 octets): fe 92 7a e2 71 31 2e 8b f0 27 5b 58 1c 54 ee f0
20 45 0d c4 ec ff aa 05 a1 a3 5d 27 51 8e 78 03<br>
key info (13 octets):
 00 10 09 74 6c 73 31 33 20 6b 65 79 00<br>
key expanded (16 octets):
 27 c6 bd c0 a3 dc ea 39 a4 73 26 d7 9b
c9 e4 ee<br>
iv info (12 octets):
 00 0c 08 74 6c 73 31 33 20 69 76 00<br>
iv expanded (12 octets):
 95 69 ec dd 4d 05 36 70 5e 9e f7 25

In [22]:
hkdf.salt(s_hs_traffic)
skey = hkdf.expand_label('key', '', 16)
siv = hkdf.expand_label('iv', '', 12)
hexprint(skey)
hexprint(siv)

[27,c6,bd,c0,a3,dc,ea,39,a4,73,26,d7,9b,c9,e4,ee,]
[95,69,ec,dd,4d,05,36,70,5e,9e,f7,25,]


### {server} construct an EncryptedExtensions handshake message:
EncryptedExtensions (44 octets): 08 00 00 28 00 26 00 0a 00 14 00
12 00 1d 00 17 00 18 00 19 01 00 01 01 01 02 01 03 01 04 00 1c
00 02 40 01 00 00 00 00 00 2a 00 00

In [23]:
ence = '08 00 00 28 00 26 00 0a 00 14 00 12 00 1d 00 17 00 18 00 19 01 00 01 01 01 02 01 03 01 04 00 1c 00 02 40 01 00 00 00 00 00 2a 00 00'
encrypted_ext = hexstr2list(ence)

### {server} calculate finished "tls13 finished":
PRK (32 octets): fe 92 7a e2 71 31 2e 8b f0 27 5b 58 1c 54 ee f0
20 45 0d c4 ec ff aa 05 a1 a3 5d 27 51 8e 78 03<br>
hash (0 octets):
 (empty)<br>
info (18 octets):
 00 20 0e 74 6c 73 31 33 20 66 69 6e 69 73 68 65
64 00<br>
expanded (32 octets): 4b b7 4c ae 7a 5d c8 91 46 04 c0 bf be 2f
0c 06 23 96 88 39 22 be c8 a1 5e 2a 9b 53 2a 5d 39 2c<br>
finished (32 octets): 48 d3 e0 e1 b3 d9 07 c6 ac ff 14 5e 16 09
03 88 c7 7b 05 c0 50 b6 34 ab 1a 88 bb d0 dd 1a 34 b2

In [24]:
hkdf.salt(s_hs_traffic)
expanded2 = hkdf.expand_label('finished', '', 32)
hexprint(expanded2)
hkdf.salt(expanded2)
hashed = sha.hash(payload + server_hello + encrypted_ext)
hexprint(hashed)
finished = hkdf.hash(hashed)
hexprint(finished)

[4b,b7,4c,ae,7a,5d,c8,91,46,04,c0,bf,be,2f,0c,06,23,96,88,39,22,be,c8,a1,5e,2a,9b,53,2a,5d,39,2c,]
[04,05,54,55,ef,74,b2,32,2b,a3,66,cb,c4,cf,e0,27,23,43,4b,37,b9,b3,67,1b,b5,a3,00,60,56,d8,f0,2f,]
[48,d3,e0,e1,b3,d9,07,c6,ac,ff,14,5e,16,09,03,88,c7,7b,05,c0,50,b6,34,ab,1a,88,bb,d0,dd,1a,34,b2,]


### {server} construct a Finished handshake message:
Finished (36 octets): 14 00 00 20 48 d3 e0 e1 b3 d9 07 c6 ac ff
14 5e 16 09 03 88 c7 7b 05 c0 50 b6 34 ab 1a 88 bb d0 dd 1a 34
b2

### {server} send handshake record:
payload (80
 octets): 08 00 00 28 00 26 00 0a 00 14 00 12 00 1d 00
17 00 18
 00 19 01 00 01 01 01 02 01 03 01 04 00 1c 00 02 40 01
00 00 00
 00 00 2a 00 00 14 00 00 20 48 d3 e0 e1 b3 d9 07 c6 ac
ff 14 5e
 16 09 03 88 c7 7b 05 c0 50 b6 34 ab 1a 88 bb d0 dd 1a
34 b2<br>
complete
 record (102
 octets):
 17 03 03 00 61 dc 48 23 7b 4b 87 9f
50 d0
 d4 d2 62 ea
 8b 47 16
 eb 40 dd c1 eb 95 7e 11 12 6e 8a 71
49 c2
 d0 12 d3 7a
 71 15 95
 7e 64 ce 30 00 8b 9e 03 23 f2 c0 5a
9c 1c
 77 b4 f3 78
 49 a6 95
 ab 25 50 60 a3 3f ee 77 0c a9 5c b8
48 6b
 fd 08 43 b8
 70 24 86
 5c a3 5c c4 1c 4e 51 5c 64 dc b1 36
9f 98
 63 5b c7 a5


In [25]:
seq = [0,0,0,0,0,0,0,0]
hexprint(skey)
hexprint(siv)
aes.key(list2int(skey))
aes.iv(list2int(siv))
aes.aad([0x17, 3, 3, 0, 0x61])
#aes.iv()#sequence
handshake_payload = encrypted_ext +[0x14,0,0,0x20] + finished + [0x16]
hexprint(handshake_payload)
print(len(handshake_payload))
aes.xor_with_iv(seq)
result = aes.encrypt(handshake_payload)
aes.xor_with_iv(seq)
hexprint(result[0])
hexprint(result[1])

[27,c6,bd,c0,a3,dc,ea,39,a4,73,26,d7,9b,c9,e4,ee]
[95,69,ec,dd,4d,05,36,70,5e,9e,f7,25]
[08,00,00,28,00,26,00,0a,00,14,00,12,00,1d,00,17,00,18,00,19,01,00,01,01,01,02,01,03,01,04,00,1c,00,02,40,01,00,00,00,00,00,2a,00,00,14,00,00,20,48,d3,e0,e1,b3,d9,07,c6,ac,ff,14,5e,16,09,03,88,c7,7b,05,c0,50,b6,34,ab,1a,88,bb,d0,dd,1a,34,b2,16]
81
[dc,48,23,7b,4b,87,9f,50,d0,d4,d2,62,ea,8b,47,16,eb,40,dd,c1,eb,95,7e,11,12,6e,8a,71,49,c2,d0,12,d3,7a,71,15,95,7e,64,ce,30,00,8b,9e,03,23,f2,c0,5a,9c,1c,77,b4,f3,78,49,a6,95,ab,25,50,60,a3,3f,ee,77,0c,a9,5c,b8,48,6b,fd,08,43,b8,70,24,86,5c,a3]
[5c,c4,1c,4e,51,5c,64,dc,b1,36,9f,98,63,5b,c7,a5]


### {server} derive secret "tls13 c ap traffic":
PRK (32 octets): e2 d3 2d 4e d6 6d d3 78 97 a0 e8 0c 84 10 75 03
ce 58 bf 8a ad 4c b5 5a 50 02 d7 7e cb 89 0e ce<br>
hash (32 octets): b0 ae ff c4 6a 2c fe 33 11 4e 6f d7 d5 1f 9f 04
b1 ca 3c 49 7d ab 08 93 4a 77 4a 9d 9a d7 db f3<br>
info (54 octets): 00 20 12 74 6c 73 31 33 20 63 20 61 70 20 74 72
61 66 66 69 63 20 b0 ae ff c4 6a 2c fe 33 11 4e 6f d7 d5 1f 9f
04 b1 ca 3c 49 7d ab 08 93 4a 77 4a 9d 9a d7 db f3<br>
expanded (32 octets): 2a bb f2 b8 e3 81 d2 3d be be 1d d2 a7 d1
6a 8b f4 84 cb 49 50 d2 3f b7 fb 7f a8 54 70 62 d9 a1

In [36]:
hexprint(master)
hkdf.salt(master)
c_ap_traffic = hkdf.derive_secret('c ap traffic', str2list(client_hello + server_h) + handshake_payload)
hexprint(c_ap_traffic)

[e2,d3,2d,4e,d6,6d,d3,78,97,a0,e8,0c,84,10,75,03,ce,58,bf,8a,ad,4c,b5,5a,50,02,d7,7e,cb,89,0e,ce,]
[90,b8,36,9f,31,99,bd,25,fb,08,1f,59,89,7c,3e,5e,a3,c6,b8,6e,5a,b1,6e,fa,ef,47,d5,17,03,9b,4d,ad,]
